In [58]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB
import pandas as pd

#gögn frá repository líklega smá bull 
dico = {0:"Rome",1:"Venice",2:"Madrid",3:"Barcelona",4:"Lisbon",5:"London",6:"Berlin",7:"Hamburg",8:"Budapest",9:"Amsterdam",10:"Paris",11:"Vienna"}
nb_cities = len(dico)

infi=10000
enjoyement = np.array([4.5, 5.9, 8.4, 5.8, 10, 2.7, 7.4, 5.2, 9.3, 7, 4.1, 6])
hostel = np.array([44, 52, 29, 24, 16, 23, 30, 25, 10, 27, 44, 36])
flights = np.array([[93, infi, infi, 105, infi, 64, 137, infi, infi, 94, 110, infi], 
                    [280, infi, infi, 260, infi, 180, 210, infi, infi, 190, 210, infi]])


nb_days = 7
budget = 1500
rest_time = 6
transit = 3
same=100000
travel_time = np.array([[same, 280, infi, infi, 260, infi, 180, 210, infi, infi, 190, 210, infi], 
                 [280, same, 239, 1045, 879, 1808, 868, 919, 984, 933, 930, 654, 732], 
                 [infi, 239, same, 1002, 836, 1765, 834, 769.0, 941, 667, 887, 611, 466], 
                 [infi, 1045, 1002, same, 150, 1005, 829, 1188, 1172, 1581, 891, 615, 1365], 
                 [260, 879, 836, 150, same, 795, 619, 978, 953, 1346, 681, 405, 1164], 
                 [infi, 1808, 1765, 1005, 795, same, 1943, 1684, 1608, 2412, 1336, 1064, 1756], 
                 [180, 868, 834, 829, 619, 1943, same, 569, 553, 1207, 251, 139, 1006], 
                 [210, 919, 769, 1188, 978, 1684, 569, same, 106, 737, 397, 532, 536], 
                 [infi, 984, 941, 1172, 953, 1608, 553, 106, same, 902, 350, 507, 701], 
                 [infi, 933, 667, 1581, 1346, 2412, 1207, 737, 902, same, 919, 797, 146], 
                 [190, 930, 887, 891, 681, 1336, 251, 397, 350, 919, same, 205, 763], 
                 [210, 654, 611, 615, 405, 1064, 139, 532, 507, 797, 205, same, 608], 
                 [infi, 732, 466, 1365, 1164, 1756, 1006, 536, 701, 146, 763, 608, same]])

In [57]:
# Create the model
m = gp.Model()

x = {}
y = {}

# Add decision variables
for day in range(nb_days):
    for city in range(nb_cities):
        x[day, city] = m.addVar(vtype=GRB.BINARY, name=f"x{day}{city}")
        y[day, city] = m.addVar(lb = 0.0, vtype=GRB.CONTINUOUS, name=f"t{day}{city}")
        
        
# Set the objective

#ákvað að margfalda enjoyment með tímanum sem er eitt á staðnum 
m.setObjective(gp.quicksum((enjoyement[city]*x[day, city]*(24-rest_time-(travel_time[city][city-1])/24) + y[day, city])
                               for day in range(nb_days) 
                               for city in range(nb_cities)), GRB.MAXIMIZE)
# Add constraints


m.addConstr(gp.quicksum(flights[0][city]*x[0, city] for city in range(nb_cities)) + 
                gp.quicksum(flights[1][city]*x[nb_days-1, city] for city in range(nb_cities)) + 
                gp.quicksum(hostel[city]*x[day, city] for day in range(nb_days) for city in range(nb_cities)) <= budget, name="Budget")

m.addConstr(gp.quicksum(x[day, city] for day in range(nb_days) for city in range(nb_cities)) == nb_days, name=f"Visit on day{day}")

# One time maximum in a city
for city in range(nb_cities):
    m.addConstr(gp.quicksum(x[day, city] for day in range(nb_days)) <= 1, name="One time max per city")

# Exactly one city per day
for day in range(nb_days):
    m.addConstr(gp.quicksum(x[day, city] for city in range(nb_cities)) == 1, name="One city per day")
for day in range(nb_days):
    for city in range(nb_cities):
        m.addConstr(y[day, city] <= (24-rest_time)*60, name=f"Visit time max on {day} in {city}")
    
for day in range(nb_days):
    for city in range(nb_cities):
        if day == 0:
            m.addConstr(y[day, city] + flights[0][city]*x[0, city] <= (24-rest_time-transit)*60)
        elif day == 6:
            m.addConstr(y[day, city] + flights[1][city]*x[0, city] <= (24-rest_time-transit)*60 )                
        else:
            m.addConstr(y[day, city] + gp.quicksum(travel_time[city][city_yest] * x[day, city] * x[day-1, city_yest] for city_yest in range(nb_cities)) <= (24-rest_time)*60)


# Optimize model
m.optimize()

if m.status == GRB.OPTIMAL:
    for day in range(nb_days):
        for city in range(nb_cities):
            if x[day, city].x > 0.1:
                time = y[day, city].x
                loc = dico[city]
                print(f"Day {day+1} in {loc} for {time/60:.1f} hours.")
        

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 129 rows, 168 columns and 468 nonzeros
Model fingerprint: 0xf06fb107
Model has 60 quadratic constraints
Variable types: 84 continuous, 84 integer (84 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [1e+02, 1e+05]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
  QRHS range       [1e+03, 1e+03]
Presolve removed 110 rows and 36 columns
Presolve time: 0.01s
Presolved: 727 rows, 780 columns, 2796 nonzeros
Variable types: 0 continuous, 780 integer (720 binary)
Found heuristic solution: objective 82927.383333

Root relaxation: objective 8.641213e+04, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |